# Import Libraries

In [1]:
import numpy as np
import pandas as pd 
import keras
from PIL import Image
import warnings
import os
import time
from glob import glob
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd, os
import xml.etree.eleTres as ET 
import matplotlib.pyplot as plt, zipfile 
from PIL import Image 
from glob import glob
from keras.layers import Input, Dense, Reshape, Flatten, Dropout,Concatenate 
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import concatenate
from keras.initializers import TruncatedNormal
from keras.callbacks import LearningRateScheduler, EarlyStopping, History

Using TensorFlow backend.


In [2]:
print(os.listdir("../input/"))

['all-dogs', 'annotation']


In [ ]:
ComputeLB = False
DogsOnly = False

import numpy as np, pandas as pd, os
import xml.etree.eleTres as ET 
import matplotlib.pyplot as plt, zipfile 
from PIL import Image 

BASE = '../input/gan/'
if not ComputeLB: BASE = '../input/'
IMAGES = os.listdir(BASE + 'all-dogs/all-dogs/')
breeds = os.listdir(BASE + 'annot/annot/') 

idxIn = 0; namesIn = []
imagesIn = np.zeros((25000,64,64,3))
 
if DogsOnly:
    for breed in breeds:
        for dog in os.listdir(BASE+'annot/annot/'+breed):
            try: img = Image.open(BASE+'all-dogs/all-dogs/'+dog+'.jpg') 
            except: continue           
            tree = ET.parse(BASE+'annot/annot/'+breed+'/'+dog)
            BASE = tree.getBASE()
            objects = BASE.findall('object') 
    idx = np.arange(idxIn)
    np.random.shuffle(idx)
    imagesIn = imagesIn[idx,:,:,:]
    namesIn = np.array(namesIn)[idx]
     
else:
    IMAGES = np.sort(IMAGES)
    np.random.seed(810)
    x = np.random.choice(np.arange(20579),10000 
     
x = np.random.randint(0,idxIn,25)
for k in range(5):
    plt.figure(figsize=(15,3))
    for j in range(5):
        plt.subplot(1,5,j+1)
        img = Image.fromarray( imagesIn[x[k*5+j],:,:,:].astype('uint8') )
        plt.axis('off')
        if not DogsOnly: plt.title(namesIn[x[k*5+j]],fontsize=11)
        else: plt.title(namesIn[x[k*5+j]].split('-')[1],fontsize=11)
        plt.imshow(img)
    plt.show()

In [7]:
def GeneratorFunction(noise_shape=(N_SI,)):
    input_layer = Input(noise_shape)
    generated = Dense(12288, activation='linear')(input_layer) 
    model = Model(inputs=input_layer,outputs = [generated,Reshape((10000,))(input_layer)])
    model.summary()
  
    return model

In [8]:
print("Generator")
model_generator = GeneratorFunction(noise_shape=(N_SI,))

Generator
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10000)        0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 12288)        122892288   input_3[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 10000)        0           input_3[0][0]                    
Total params: 122,892,288
Trainable params: 122,892,288
Non-trainable params: 0
__________________________________________________________________________________________________


# Train the Discriminator

In [9]:
# TRAINING DATA
train_y = (imagesIn[:10000,:,:,:]/255.).reshape((-1,12288))
train_X = np.zeros((10000,10000))
for i in range(10000): train_X[i,i] = 1
zeros = np.zeros((10000,12288))

# ---------------------
#  Train Discriminator
# ---------------------

lr = 0.5
for k in range(5):
    LR_Scheduler = LearningRateScheduler(lambda x: lr)
    h = model_discriminator.fit([zeros,train_X], train_y, epochs = 100,batch_size = BATCH_SIZE, callbacks=[LR_Scheduler], verbose=0)
    print('Epoch',(k+1)*10,'/50 - loss =',h.history['loss'][-1] )
    if h.history['loss'][-1]<0.533: lr = 0.1

Epoch 10 /50 - loss = 0.5324619684696198
Epoch 20 /50 - loss = 0.5324463943481446
Epoch 30 /50 - loss = 0.5324408501625061
Epoch 40 /50 - loss = 0.5324368857383728
Epoch 50 /50 - loss = 0.5324339078426361


In [10]:
del train_X, train_y, imagesIn

# Build the GAN

In [12]:
# BUILD GENERATIVE ADVERSARIAL NETWORK
model_discriminator.trainable = False #discriminator is not trainable for GANs
z = Input(shape=(N_SI,))
img = model_generator(z)
real = model_discriminator(img)

# COMPILE GAN
gan = Model(z, real)
gan.get_layer('model_1').get_layer('layer_1').set_weights([np.array([[[[-1 ]]],[[[255.]]]])])
gan.compile(optimizer=Adam(5), loss='mean_squared_error')

# DISPLAY ARCHITECTURE
print("Model created based on Discriminator and Generator")
gan.summary()

Model created based on Discriminator and Generator
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 10000)        0                                            
__________________________________________________________________________________________________
model_2 (Model)                 [(None, 12288), (Non 122892288   input_4[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 12288)        122892290   model_2[1][0]                    
                                                                 model_2[1][1]                    
Total params: 245,784,578
Trainable params: 122,892,288
Non-trainable params: 122,892,290
___________________________________________________________________

# Train the GAN

In [13]:
train = np.zeros((10000,10000))
for i in range(10000): train[i,i] = 1
zeros = np.zeros((10000,12288))

Steps_per_epoch = 50

print("Training begins... Total steps per epoch: {}".format(Steps_per_epoch ))


Training begins... Total steps per epoch: 50


# Submit to Kaggle

In [15]:
class DogGenerator:
    index = 0   
    def getDog(self,seed):
        xx = np.zeros((10000))
        xx[self.index] = 0.999
        xx[np.random.randint(10000)] = 0.001
        img = model_generator.predict(xx.reshape((-1,10000)))[0].reshape((64,64,3))
        self.index = (self.index+1)%10000
        return Image.fromarray( img.astype('uint8') ) 